In [4]:
! pip install --upgrade google-cloud-bigquery-storage

In [5]:
import google, pathlib, shutil, urllib, zipfile as zf, numpy as np, pandas as pd, geopandas as gpd
from google.cloud import aiplatform, bigquery
from shapely.ops import orient
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')

proj_id = 'cmat-315920'
root_path = '/home/jupyter'
root_path = pathlib.Path(root_path)
raw_data_path = root_path / 'raw_data'
cred, proj = google.auth.default(scopes=["https://www.googleapis.com/auth/cloud-platform"])
bqclient = bigquery.Client(credentials = cred, project = proj)

def read_file(zipfile, f, **kwargs):
    file = zipfile.extract(f)
    df = pd.read_csv(file, dtype=str, **kwargs)
    df.columns = [str(x).lower() for x in df.columns]
    return df

def get_states():
    qry = f"""
    select
        state_fips_code as fips
        , state_postal_abbreviation as abbr
        , state_name as name
    from
        bigquery-public-data.census_utility.fips_codes_states
    where
        state_fips_code <= '56'
    """
    return bqclient.query(qry).result().to_dataframe()

def get_data(name, url, state, yr=2020, overwrite=False):
    print(f"Get {state['name']} {yr} {name}", end='...' )
    path = f"{name}/{yr}/{state['abbr']}"
    parquetfile = raw_data_path / f"{path}/{path.replace('/', '_')}.parquet"
    zipfile = parquetfile.with_suffix('.zip')
    path = parquetfile.parent
    
    if overwrite:
        shutil.rmtree(path, ignore_errors=True)
    path.mkdir(parents=True, exist_ok=True)
    %cd {path}
    try:
        df = gpd.read_parquet(parquetfile)
        print(f'exisiting geoparquet found {parquetfile}')
    except:
        try:
            df = pd.read_parquet(parquetfile)
            print(f'exisiting parquet found {parquetfile}')
        except:
            df = None
            
            try:
                zipfile = zf.ZipFile(zipfile)
                print('already have zip')
            except:
                try:
                    print(f'\nfetching zip from {url}', end = '...')
                    zipfile = zf.ZipFile(urllib.request.urlretrieve(url, zipfile)[0])
                    print('success')
                except urllib.error.HTTPError:
                    print('\n\nFAILED - BAD URL\n\n')
                    zipfile = None
    return df, parquetfile, zipfile

def write_data(df, file):
    df.to_parquet(file)
    table_id = f"{proj_id}.redistricting.{file.stem}"
    job = bqclient.load_table_from_dataframe(df, table_id)
    job.result()  # Wait for the job to complete.

states = get_states()
states = states.iloc[[39,43]]
overwrite = False
# overwrite = True

In [9]:
abbr = "RI"
file = raw_data_path / f'census/2010/{abbr}/{abbr.lower()}geo2010.pl'
file.is_file()
# make a loop
# how to read file read one line at a time and only get the row that has block 22
width = np.array([1, 6, 2, 3, 2, 3, 2, 7, 1, 1, 2, 3, 2, 2, 5, 2, 2, 5, 2, 2, 6, 1, 4,  2, 5, 2, 2, 4, 5, 2, 1, 3, 5, 2, 6, 1, 5, 2, 5, 2, 5, 3, 5, 2, 5, 3, 1, 1, 5, 2, 1, 1, 2, 3, 3, 6, 1, 3, 5, 5, 2, 5, 5, 5, 14,   14, 90, 1, 1, 9, 9, 11, 12, 2, 1, 6, 5, 8, 8, 8, 8, 8, 8, 8, 8, 8, 2, 2, 2, 3, 3, 3, 3, 3, 3, 2, 2, 2, 1, 1, 5, 18])
np.cumsum(width)

array([  1,   7,   9,  12,  14,  17,  19,  26,  27,  28,  30,  33,  35,
        37,  42,  44,  46,  51,  53,  55,  61,  62,  66,  68,  73,  75,
        77,  81,  86,  88,  89,  92,  97,  99, 105, 106, 111, 113, 118,
       120, 125, 128, 133, 135, 140, 143, 144, 145, 150, 152, 153, 154,
       156, 159, 162, 168, 169, 172, 177, 182, 184, 189, 194, 199, 213,
       227, 317, 318, 319, 328, 337, 348, 360, 362, 363, 369, 374, 382,
       390, 398, 406, 414, 422, 430, 438, 446, 448, 450, 452, 455, 458,
       461, 464, 467, 470, 472, 474, 476, 477, 478, 483, 501])

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

def get_census(state, yr, overwrite=False, nrows=None):
    yrs = [2010]
    assert yr in yrs, f'yr must be in {yrs} - got {yr}'
    name = f'census'
    url = f"https://www2.census.gov/programs-surveys/decennial/{yr}/data/01-Redistricting_File--PL_94-171/{state['name'].replace(' ', '_')}/{state['abbr'].lower()}{yr}.pl.zip"
    df, parquetfile, zipfile = get_data(name, url, state, yr, overwrite)

    if df is None and zipfile is not None:
        L = []
        if yr == 2010:
            idx = ['fileid', 'stusab', 'chariter', 'logrecno']
            for f in zipfile.namelist():
                if f[-2:] == 'pl':
                   
                    if f[2:5] == 'geo':
                        widths = [6, 2, 3, 2, 3, 2, 7, 1, 1, 2, 3, 2, 2, 5, 2, 2, 5, 2, 2, 6, 1, 4, 2, 5, 2, 2, 4, 5, 2, 1, 3, 5, 2, 6, 1, 5, 2, 5, 2, 5, 3, 5, 2, 5, 3, 1, 1, 5, 2, 1, 1, 2, 3, 3, 6, 1, 3, 5, 5, 2, 5, 5, 5, 14, 14, 90, 1, 1, 9, 9, 11, 12, 2, 1, 6, 5, 8, 8, 8, 8, 8, 8, 8, 8, 8, 2, 2, 2, 3, 3, 3, 3, 3, 3, 2, 2, 2, 1, 1, 5, 18]
                        names = ['fileid', 'stusab', 'sumlev', 'geocomp', 'chariter', 'cifsn', 'logrecno', 'region', 'division', 'state', 'county', 'countycc', 'countysc', 'cousub', 'cousubcc', 'cousubsc', 'place', 'placecc', 'placesc', 'tract', 'blkgrp', 'block', 'iuc', 'concit', 'concitcc', 'concitsc', 'aianhh', 'aianhhfp', 'aianhhcc', 'aihhtli', 'aitsce', 'aits', 'aitscc', 'ttract', 'tblkgrp', 'anrc', 'anrccc', 'cbsa', 'cbsasc', 'metdiv', 'csa', 'necta', 'nectasc', 'nectadiv', 'cnecta', 'cbsapci', 'nectapci', 'ua', 'uasc', 'uatype', 'ur', 'cd', 'sldu', 'sldl', 'vtd', 'vtdi', 'reserve2', 'zcta5', 'submcd', 'submcdcc', 'sdelm', 'sdsec', 'sduni', 'arealand', 'areawatr', 'name', 'funcstat', 'gcuni', 'pop100', 'hu100', 'intptlat', 'intptlon', 'lsadc', 'partflag', 'reserve3', 'uga', 'statens', 'countyns', 'cousubns', 'placens', 'concitns', 'aianhhns', 'aitsns', 'anrcns', 'submcdns', 'cd113', 'cd114', 'cd115', 'sldu2', 'sldu3', 'sldu4', 'sldl2', 'sldl3', 'sldl4', 'aianhhsc', 'csasc', 'cnectasc', 'memi', 'nmemi', 'puma', 'reserved']
                        df = pd.read_fwf(zipfile.extract(f), header=None, widths=widths, dtype=str, nrows=nrows).fillna('')
                        df = df.query('block != ""')
                    elif f[6] == '1':
                        names = ['fileid', 'stusab', 'chariter', 'cifsn', 'logrecno', 'total', 'population of one race', 'white alone', 'black or african american alone', 'american indian and alaska native alone', 'asian alone', 'native hawaiian and other pacific islander alone', 'some other race alone', 'population of two or more races', 'population of two races', 'white; black or african american', 'white; american indian and alaska native', 'white; asian', 'white; native hawaiian and other pacific islander', 'white; some other race', 'black or african american; american indian and alaska native', 'black or african american; asian', 'black or african american; native hawaiian and other pacific islander', 'black or african american; some other race', 'american indian and alaska native; asian', 'american indian and alaska native; native hawaiian and other pacific islander', 'american indian and alaska native; some other race', 'asian; native hawaiian and other pacific islander', 'asian; some other race', 'native hawaiian and other pacific islander; some other race', 'population of three races', 'white; black or african american; american indian and alaska native', 'white; black or african american; asian', 'white; black or african american; native hawaiian and other pacific islander', 'white; black or african american; some other race', 'white; american indian and alaska native; asian', 'white; american indian and alaska native; native hawaiian and other pacific islander', 'white; american indian and alaska native; some other race', 'white; asian; native hawaiian and other pacific islander', 'white; asian; some other race', 'white; native hawaiian and other pacific islander; some other race', 'black or african american; american indian and alaska native; asian', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander', 'black or african american; american indian and alaska native; some other race', 'black or african american; asian; native hawaiian and other pacific islander', 'black or african american; asian; some other race', 'black or african american; native hawaiian and other pacific islander; some other race', 'american indian and alaska native; asian; native hawaiian and other pacific islander', 'american indian and alaska native; asian; some other race', 'american indian and alaska native; native hawaiian and other pacific islander; some other race', 'asian; native hawaiian and other pacific islander; some other race', 'population of four races', 'white; black or african american; american indian and alaska native; asian', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander', 'white; black or african american; american indian and alaska native; some other race', 'white; black or african american; asian; native hawaiian and other pacific islander', 'white; black or african american; asian; some other race', 'white; black or african american; native hawaiian and other pacific islander; some other race', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander', 'white; american indian and alaska native; asian; some other race', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race', 'white; asian; native hawaiian and other pacific islander; some other race', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander', 'black or african american; american indian and alaska native; asian; some other race', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race', 'black or african american; asian; native hawaiian and other pacific islander; some other race', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'population of five races', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander', 'white; black or african american; american indian and alaska native; asian; some other race', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'population of six races', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race', 'total HL', 'hispanic or latino HL', 'not hispanic or latino HL', 'population of one race HL', 'white alone HL', 'black or african american alone HL', 'american indian and alaska native alone HL', 'asian alone HL', 'native hawaiian and other pacific islander alone HL', 'some other race alone HL', 'population of two or more races HL', 'population of two races HL', 'white; black or african american HL', 'white; american indian and alaska native HL', 'white; asian HL', 'white; native hawaiian and other pacific islander HL', 'white; some other race HL', 'black or african american; american indian and alaska native HL', 'black or african american; asian HL', 'black or african american; native hawaiian and other pacific islander HL', 'black or african american; some other race HL', 'american indian and alaska native; asian HL', 'american indian and alaska native; native hawaiian and other pacific islander HL', 'american indian and alaska native; some other race HL', 'asian; native hawaiian and other pacific islander HL', 'asian; some other race HL', 'native hawaiian and other pacific islander; some other race HL', 'population of three races HL', 'white; black or african american; american indian and alaska native HL', 'white; black or african american; asian HL', 'white; black or african american; native hawaiian and other pacific islander HL', 'white; black or african american; some other race HL', 'white; american indian and alaska native; asian HL', 'white; american indian and alaska native; native hawaiian and other pacific islander HL', 'white; american indian and alaska native; some other race HL', 'white; asian; native hawaiian and other pacific islander HL', 'white; asian; some other race HL', 'white; native hawaiian and other pacific islander; some other race HL', 'black or african american; american indian and alaska native; asian HL', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander HL', 'black or african american; american indian and alaska native; some other race HL', 'black or african american; asian; native hawaiian and other pacific islander HL', 'black or african american; asian; some other race HL', 'black or african american; native hawaiian and other pacific islander; some other race HL', 'american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'american indian and alaska native; asian; some other race HL', 'american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'asian; native hawaiian and other pacific islander; some other race HL', 'population of four races HL', 'white; black or african american; american indian and alaska native; asian HL', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander HL', 'white; black or african american; american indian and alaska native; some other race HL', 'white; black or african american; asian; native hawaiian and other pacific islander HL', 'white; black or african american; asian; some other race HL', 'white; black or african american; native hawaiian and other pacific islander; some other race HL', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'white; american indian and alaska native; asian; some other race HL', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'white; asian; native hawaiian and other pacific islander; some other race HL', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'black or african american; american indian and alaska native; asian; some other race HL', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'black or african american; asian; native hawaiian and other pacific islander; some other race HL', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL', 'population of five races HL', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL', 'white; black or african american; american indian and alaska native; asian; some other race HL', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race HL', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL', 'population of six races HL', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL']
                        df = read_file(zipfile, f          , header=None, nrows=nrows)
                    elif f[6] == '2':
                        names = ['fileid', 'stusab', 'chariter', 'cifsn', 'logrecno', 'total 18', 'population of one race 18', 'white alone 18', 'black or african american alone 18', 'american indian and alaska native alone 18', 'asian alone 18', 'native hawaiian and other pacific islander alone 18', 'some other race alone 18', 'population of two or more races 18', 'population of two races 18', 'white; black or african american 18', 'white; american indian and alaska native 18', 'white; asian 18', 'white; native hawaiian and other pacific islander 18', 'white; some other race 18', 'black or african american; american indian and alaska native 18', 'black or african american; asian 18', 'black or african american; native hawaiian and other pacific islander 18', 'black or african american; some other race 18', 'american indian and alaska native; asian 18', 'american indian and alaska native; native hawaiian and other pacific islander 18', 'american indian and alaska native; some other race 18', 'asian; native hawaiian and other pacific islander 18', 'asian; some other race 18', 'native hawaiian and other pacific islander; some other race 18', 'population of three races 18', 'white; black or african american; american indian and alaska native 18', 'white; black or african american; asian 18', 'white; black or african american; native hawaiian and other pacific islander 18', 'white; black or african american; some other race 18', 'white; american indian and alaska native; asian 18', 'white; american indian and alaska native; native hawaiian and other pacific islander 18', 'white; american indian and alaska native; some other race 18', 'white; asian; native hawaiian and other pacific islander 18', 'white; asian; some other race 18', 'white; native hawaiian and other pacific islander; some other race 18', 'black or african american; american indian and alaska native; asian 18', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander 18', 'black or african american; american indian and alaska native; some other race 18', 'black or african american; asian; native hawaiian and other pacific islander 18', 'black or african american; asian; some other race 18', 'black or african american; native hawaiian and other pacific islander; some other race 18', 'american indian and alaska native; asian; native hawaiian and other pacific islander 18', 'american indian and alaska native; asian; some other race 18', 'american indian and alaska native; native hawaiian and other pacific islander; some other race 18', 'asian; native hawaiian and other pacific islander; some other race 18', 'population of four races 18', 'white; black or african american; american indian and alaska native; asian 18', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander 18', 'white; black or african american; american indian and alaska native; some other race 18', 'white; black or african american; asian; native hawaiian and other pacific islander 18', 'white; black or african american; asian; some other race 18', 'white; black or african american; native hawaiian and other pacific islander; some other race 18', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander 18', 'white; american indian and alaska native; asian; some other race 18', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race 18', 'white; asian; native hawaiian and other pacific islander; some other race 18', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander 18', 'black or african american; american indian and alaska native; asian; some other race 18', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race 18', 'black or african american; asian; native hawaiian and other pacific islander; some other race 18', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race 18', 'population of five races 18', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander 18', 'white; black or african american; american indian and alaska native; asian; some other race 18', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race 18', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race 18', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race 18', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race 18', 'population of six races 18', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race 18', 'total HL18', 'hispanic or latino HL18', 'not hispanic or latino HL18', 'population of one race HL18', 'white alone HL18', 'black or african american alone HL18', 'american indian and alaska native alone HL18', 'asian alone HL18', 'native hawaiian and other pacific islander alone HL18', 'some other race alone HL18', 'population of two or more races HL18', 'population of two races HL18', 'white; black or african american HL18', 'white; american indian and alaska native HL18', 'white; asian HL18', 'white; native hawaiian and other pacific islander HL18', 'white; some other race HL18', 'black or african american; american indian and alaska native HL18', 'black or african american; asian HL18', 'black or african american; native hawaiian and other pacific islander HL18', 'black or african american; some other race HL18', 'american indian and alaska native; asian HL18', 'american indian and alaska native; native hawaiian and other pacific islander HL18', 'american indian and alaska native; some other race HL18', 'asian; native hawaiian and other pacific islander HL18', 'asian; some other race HL18', 'native hawaiian and other pacific islander; some other race HL18', 'population of three races HL18', 'white; black or african american; american indian and alaska native HL18', 'white; black or african american; asian HL18', 'white; black or african american; native hawaiian and other pacific islander HL18', 'white; black or african american; some other race HL18', 'white; american indian and alaska native; asian HL18', 'white; american indian and alaska native; native hawaiian and other pacific islander HL18', 'white; american indian and alaska native; some other race HL18', 'white; asian; native hawaiian and other pacific islander HL18', 'white; asian; some other race HL18', 'white; native hawaiian and other pacific islander; some other race HL18', 'black or african american; american indian and alaska native; asian HL18', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander HL18', 'black or african american; american indian and alaska native; some other race HL18', 'black or african american; asian; native hawaiian and other pacific islander HL18', 'black or african american; asian; some other race HL18', 'black or african american; native hawaiian and other pacific islander; some other race HL18', 'american indian and alaska native; asian; native hawaiian and other pacific islander HL18', 'american indian and alaska native; asian; some other race HL18', 'american indian and alaska native; native hawaiian and other pacific islander; some other race HL18', 'asian; native hawaiian and other pacific islander; some other race HL18', 'population of four races HL18', 'white; black or african american; american indian and alaska native; asian HL18', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander HL18', 'white; black or african american; american indian and alaska native; some other race HL18', 'white; black or african american; asian; native hawaiian and other pacific islander HL18', 'white; black or african american; asian; some other race HL18', 'white; black or african american; native hawaiian and other pacific islander; some other race HL18', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander HL18', 'white; american indian and alaska native; asian; some other race HL18', 'white; american indian and alaska native; native hawaiian and other pacific islander; some other race HL18', 'white; asian; native hawaiian and other pacific islander; some other race HL18', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL18', 'black or african american; american indian and alaska native; asian; some other race HL18', 'black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL18', 'black or african american; asian; native hawaiian and other pacific islander; some other race HL18', 'american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL18', 'population of five races HL18', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander HL18', 'white; black or african american; american indian and alaska native; asian; some other race HL18', 'white; black or african american; american indian and alaska native; native hawaiian and other pacific islander; some other race HL18', 'white; black or african american; asian; native hawaiian and other pacific islander; some other race HL18', 'white; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL18', 'black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL18', 'population of six races HL18', 'white; black or african american; american indian and alaska native; asian; native hawaiian and other pacific islander; some other race HL18', 'housing total', 'housing occupied', 'housing vacant']
                        df = read_file(zipfile, f          , header=None, nrows=nrows)
                    df.columns = names
                    L.append(df.drop(columns='cifsn').set_index(idx))
            df = pd.concat(L, axis=1).reset_index()
            df.columns = [c.replace(' ', '_').replace(';', '_') for c in df.columns]
        display(df.head(3))
        write_data(df, parquetfile)
    return df

for i, state in states.iterrows():
    for yr in [2010]:#, 2020]:
        census = get_census(state, yr, overwrite, 100)

Get Rhode Island 2010 census.../home/jupyter/raw_data/census/2010/RI

fetching zip from https://www2.census.gov/programs-surveys/decennial/2010/data/01-Redistricting_File--PL_94-171/Rhode_Island/ri2010.pl.zip...success


,fileid,stusab,chariter,logrecno,sumlev,geocomp,region,division,state,county,countycc,countysc,cousub,cousubcc,cousubsc,place,placecc,placesc,tract,blkgrp,block,iuc,concit,concitcc,concitsc,aianhh,aianhhfp,aianhhcc,aihhtli,aitsce,aits,aitscc,ttract,tblkgrp,anrc,anrccc,cbsa,cbsasc,metdiv,csa,necta,nectasc,nectadiv,cnecta,cbsapci,nectapci,ua,uasc,uatype,ur,cd,sldu,sldl,vtd,vtdi,reserve2,zcta5,submcd,submcdcc,sdelm,sdsec,sduni,arealand,areawatr,name,funcstat,gcuni,pop100,hu100,intptlat,intptlon,lsadc,partflag,reserve3,uga,statens,countyns,cousubns,placens,concitns,aianhhns,aitsns,anrcns,submcdns,cd113,cd114,cd115,sldu2,sldu3,sldu4,sldl2,sldl3,sldl4,aianhhsc,csasc,cnectasc,memi,nmemi,puma,reserved,total,population_of_one_race,white_alone,black_or_african_american_alone,american_indian_and_alaska_native_alone,asian_alone,native_hawaiian_and_other_pacific_islander_alone,some_other_race_alone,population_of_two_or_more_races,population_of_two_races,white__black_or_african_american,white__american_indian_and_alaska_native,white__asian,white__native_hawaiian_and_other_pacific_islander,white__some_other_race,black_or_african_american__american_indian_and_alaska_native,black_or_african_american__asian,black_or_african_american__native_hawaiian_and_other_pacific_islander,black_or_african_american__some_other_race,american_indian_and_alaska_native__asian,american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander,american_indian_and_alaska_native__some_other_race,asian__native_hawaiian_and_other_pacific_islander,asian__some_other_race,native_hawaiian_and_other_pacific_islander__some_other_race,population_of_three_races,white__black_or_african_american__american_indian_and_alaska_native,white__black_or_african_american__asian,white__black_or_african_american__native_hawaiian_and_other_pacific_islander,white__black_or_african_american__some_other_race,white__american_indian_and_alaska_native__asian,white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander,white__american_indian_and_alaska_native__some_other_race,white__asian__native_hawaiian_and_other_pacific_islander,white__asian__some_other_race,white__native_hawaiian_and_other_pacific_islander__some_other_race,black_or_african_american__american_indian_and_alaska_native__asian,black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander,black_or_african_american__american_indian_and_alaska_native__some_other_race,black_or_african_american__asian__native_hawaiian_and_other_pacific_islander,black_or_african_american__asian__some_other_race,black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race,american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander,american_indian_and_alaska_native__asian__some_other_race,american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race,asian__native_hawaiian_and_other_pacific_islander__some_other_race,population_of_four_races,white__black_or_african_american__american_indian_and_alaska_native__asian,white__black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander,white__black_or_african_american__american_indian_and_alaska_native__some_other_race,white__black_or_african_american__asian__native_hawaiian_and_other_pacific_islander,white__black_or_african_american__asian__some_other_race,white__black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race,white__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander,white__american_indian_and_alaska_native__asian__some_other_race,white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race,white__asian__native_hawaiian_and_other_pacific_islander__some_other_race,black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander,black_or_african_ameri

Get Texas 2010 census.../home/jupyter/raw_data/census/2010/TX

fetching zip from https://www2.census.gov/programs-surveys/decennial/2010/data/01-Redistricting_File--PL_94-171/Texas/tx2010.pl.zip...success


,fileid,stusab,chariter,logrecno,sumlev,geocomp,region,division,state,county,countycc,countysc,cousub,cousubcc,cousubsc,place,placecc,placesc,tract,blkgrp,block,iuc,concit,concitcc,concitsc,aianhh,aianhhfp,aianhhcc,aihhtli,aitsce,aits,aitscc,ttract,tblkgrp,anrc,anrccc,cbsa,cbsasc,metdiv,csa,necta,nectasc,nectadiv,cnecta,cbsapci,nectapci,ua,uasc,uatype,ur,cd,sldu,sldl,vtd,vtdi,reserve2,zcta5,submcd,submcdcc,sdelm,sdsec,sduni,arealand,areawatr,name,funcstat,gcuni,pop100,hu100,intptlat,intptlon,lsadc,partflag,reserve3,uga,statens,countyns,cousubns,placens,concitns,aianhhns,aitsns,anrcns,submcdns,cd113,cd114,cd115,sldu2,sldu3,sldu4,sldl2,sldl3,sldl4,aianhhsc,csasc,cnectasc,memi,nmemi,puma,reserved,total,population_of_one_race,white_alone,black_or_african_american_alone,american_indian_and_alaska_native_alone,asian_alone,native_hawaiian_and_other_pacific_islander_alone,some_other_race_alone,population_of_two_or_more_races,population_of_two_races,white__black_or_african_american,white__american_indian_and_alaska_native,white__asian,white__native_hawaiian_and_other_pacific_islander,white__some_other_race,black_or_african_american__american_indian_and_alaska_native,black_or_african_american__asian,black_or_african_american__native_hawaiian_and_other_pacific_islander,black_or_african_american__some_other_race,american_indian_and_alaska_native__asian,american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander,american_indian_and_alaska_native__some_other_race,asian__native_hawaiian_and_other_pacific_islander,asian__some_other_race,native_hawaiian_and_other_pacific_islander__some_other_race,population_of_three_races,white__black_or_african_american__american_indian_and_alaska_native,white__black_or_african_american__asian,white__black_or_african_american__native_hawaiian_and_other_pacific_islander,white__black_or_african_american__some_other_race,white__american_indian_and_alaska_native__asian,white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander,white__american_indian_and_alaska_native__some_other_race,white__asian__native_hawaiian_and_other_pacific_islander,white__asian__some_other_race,white__native_hawaiian_and_other_pacific_islander__some_other_race,black_or_african_american__american_indian_and_alaska_native__asian,black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander,black_or_african_american__american_indian_and_alaska_native__some_other_race,black_or_african_american__asian__native_hawaiian_and_other_pacific_islander,black_or_african_american__asian__some_other_race,black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race,american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander,american_indian_and_alaska_native__asian__some_other_race,american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race,asian__native_hawaiian_and_other_pacific_islander__some_other_race,population_of_four_races,white__black_or_african_american__american_indian_and_alaska_native__asian,white__black_or_african_american__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander,white__black_or_african_american__american_indian_and_alaska_native__some_other_race,white__black_or_african_american__asian__native_hawaiian_and_other_pacific_islander,white__black_or_african_american__asian__some_other_race,white__black_or_african_american__native_hawaiian_and_other_pacific_islander__some_other_race,white__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander,white__american_indian_and_alaska_native__asian__some_other_race,white__american_indian_and_alaska_native__native_hawaiian_and_other_pacific_islander__some_other_race,white__asian__native_hawaiian_and_other_pacific_islander__some_other_race,black_or_african_american__american_indian_and_alaska_native__asian__native_hawaiian_and_other_pacific_islander,black_or_african_ameri

In [6]:
abbr = "RI"
file = raw_data_path / f'census/2010/{abbr}/{abbr.lower()}geo2010.pl'
file.is_file()


True

In [3]:
def get_crosswalks(state, overwrite=False):
    name = 'crosswalks'
    url = f"https://www2.census.gov/geo/docs/maps-data/data/rel2020/t10t20/TAB2010_TAB2020_ST{state['fips']}.zip"
    yr = 2020
    df, parquetfile, zipfile = get_data(name, url, state, yr, overwrite)

    if df is None and zipfile is not None:
        for f in zipfile.namelist():
            if f[-3:] == 'txt':
                df = read_file(zipfile, f, sep='|')
                for yr in [2010, 2020]:
                    df[f'geoid_{yr}'] = df[f'state_{yr}'].str.rjust(2,'0') + df[f'county_{yr}'].str.rjust(3,'0') + df[f'tract_{yr}'].str.rjust(6,'0') + df[f'blk_{yr}'].str.rjust(4,'0')
                df = df[['geoid_2010', 'geoid_2020']]
                display(df.head(2))
        write_data(df, parquetfile)
    return df
    
for i, state in states.iterrows():
    crosswalks = get_crosswalks(state, overwrite)

Get Rhode Island 2020 crosswalks.../home/jupyter/raw_data/crosswalks/2020/RI

fetching zip from https://www2.census.gov/geo/docs/maps-data/data/rel2020/t10t20/TAB2010_TAB2020_ST44.zip...success


,geoid_2010,geoid_2020
0,090117052003000,440090510001039
1,250056303002046,440070114041025


Get Texas 2020 crosswalks.../home/jupyter/raw_data/crosswalks/2020/TX

fetching zip from https://www2.census.gov/geo/docs/maps-data/data/rel2020/t10t20/TAB2010_TAB2020_ST48.zip...success


,geoid_2010,geoid_2020
0,050910209001225,480370112012022
1,050910210007000,480679501013066


In [5]:
def get_assignments(state, yr, overwrite=False):
    yrs = [2020, 2010]
    assert yr in yrs, f'yr must be in {yrs} - got {yr}'

    name = 'assignments'
    url = f"https://www2.census.gov/geo/docs/maps-data/data/baf"
    if yr == 2020:
        url += '2020'
    url += f"/BlockAssign_ST{state['fips']}_{state['abbr']}.zip"
    df, parquetfile, zipfile = get_data(name, url, state, yr, overwrite)

    if df is None and zipfile is not None:
        L = []
        for f in zipfile.namelist():
            col = f.lower().split('_')[-1][:-4]
            
            if f[-3:] == 'txt' and col != 'aiannh':
                if yr == 2020:
                    sep = '|'
                else:
                    sep = ','
                df = read_file(zipfile, f, sep=sep)
                if col == 'vtd':
                    df['countyfp'] = df['countyfp'] + df['district'].str[-4:]
                    col = 'cntyvtd'
                df = df.iloc[:,:2]
                df.columns = ['geoid', col]
                L.append(df.set_index('geoid'))
        df = pd.concat(L, axis=1).reset_index()
        write_data(df, parquetfile)
    return df

for i, state in states.iterrows():
    for yr in [2020, 2010]:
        assignments = get_assignments(state, yr, overwrite)
        display(assignments.head(2))

Get Rhode Island 2020 assignments.../home/jupyter/raw_data/assignments/2020/RI
exisiting parquet found /home/jupyter/raw_data/assignments/2020/RI/assignments_2020_RI.parquet


,geoid,cdp,mcd,cntyvtd,cd,sldu,sldl,sdsec,sdelm,sduni
0,440010301001000,None,001,0010101,01,032,066,None,None,00030
1,440010301001001,None,001,0010101,01,032,066,None,None,00030


Get Rhode Island 2010 assignments.../home/jupyter/raw_data/assignments/2010/RI

fetching zip from https://www2.census.gov/geo/docs/maps-data/data/baf/BlockAssign_ST44_RI.zip...success


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/_pandas_helpers.py:380: UserWarning: Pyarrow could not determine the type of columns: cntyvtd.
  ", ".join(field.name for field in unknown_type_fields)


BadRequest: 400 Provided Schema does not match Table cmat-315920:redistricting.assignments_2010_RI. Cannot add fields (field: cntyvtd)

In [ ]:
quest countresolution Dr. potter

In [3]:
def get_elections_TX(overwrite=False):
    name = 'elections'
    state = states.query('abbr=="TX"').iloc[0]
    yr = 2020
    url = f'https://data.capitol.texas.gov/dataset/aab5e1e5-d585-4542-9ae8-1108f45fce5b/resource/253f5191-73f3-493a-9be3-9e8ba65053a2/download/{yr}-general-vtd-election-data.zip'
    df, parquetfile, zipfile = get_data(name, url, state, yr, overwrite)

    if df is None and zipfile is not None:
        L = []
        for f in zipfile.namelist():
            w = f.split('_')
            if w.pop(-1) == 'Returns.csv':
                df = read_file(zipfile, f)
                df['cntyvtd'] = df['cntyvtd'].str.rjust(7, '0')
                df['yr'] = w.pop(0)
                df['race'] = '_'.join(w)
                L.append(df)
        df = pd.concat(L, axis=0, ignore_index=True).astype({'votes':int, 'yr':int})
        display(df.head(5))
        write_data(df, parquetfile)
    return df
elections = get_elections_TX(overwrite)
display(elections.head(2))

Get Texas 2020 elections.../home/jupyter/raw_data/elections/2020/TX

fetching zip from https://data.capitol.texas.gov/dataset/aab5e1e5-d585-4542-9ae8-1108f45fce5b/resource/253f5191-73f3-493a-9be3-9e8ba65053a2/download/2020-general-vtd-election-data.zip...success


,county,fips,vtd,cntyvtd,office,name,party,incumbent,votes,yr,race
0,Anderson,1,0001,0010001,President,Ely,D,N,2,2012,Democratic_Primary_Election
1,Anderson,1,0002,0010002,President,Ely,D,N,5,2012,Democratic_Primary_Election
2,Anderson,1,0003,0010003,President,Ely,D,N,1,2012,Democratic_Primary_Election
3,Anderson,1,0004,0010004,President,Ely,D,N,2,2012,Democratic_Primary_Election
4,Anderson,1,0005,0010005,President,Ely,D,N,1,2012,Democratic_Primary_Election


,county,fips,vtd,cntyvtd,office,name,party,incumbent,votes,yr,race
0,Anderson,1,0001,0010001,President,Ely,D,N,2,2012,Democratic_Primary_Election
1,Anderson,1,0002,0010002,President,Ely,D,N,5,2012,Democratic_Primary_Election


In [ ]:
def get_shapefiles(state, yr, level, overwrite=False):
    yrs = [2020, 2010]
    assert yr in yrs, f'yr must be in {yrs} - got {yr}'
    levels = ['tract', 'bg', 'tabblock']
    assert level in levels, f'level must be in {levels} - got {level}'
    
    name = f'shapefiles_{level}'
    url = f"https://www2.census.gov/geo/tiger/TIGER{yr}/{level.upper()}"
    if yr == 2010:
        url += '/2010'
    elif yr == 2020 and level == 'tabblock':
        url += '20'
    url += f"/tl_{yr}_{state['fips']}_{level}{str(yr)[-2:]}"
    if yr == 2020 and level in ['tract', 'bg']:
        url = url[:-2]
    url += '.zip'
    df, parquetfile, zipfile = get_data(name, url, state, yr, overwrite)

    if df is None and zipfile is not None:
        path = parquetfile.parent
        zipfile.extractall(path)
        df = gpd.read_file(path)
        df.columns = [x[:-2].lower() if x[-2:].isnumeric() else x.lower() for x in df.columns]
        df = df[['geoid', 'aland', 'geometry']]
        df['geometry'] = df['geometry'].apply(lambda p: orient(p, -1))
        write_data(df.to_wkt(), parquetfile)
    return df
    
for i, state in states.iterrows():
    for yr in [2020, 2010]:
        for level in ['tract', 'bg']:#, 'tabblock']:
            geo = get_shapefiles(state, yr, level, overwrite)

Get Rhode Island 2020 shapefiles_tract.../home/jupyter/raw_data/shapefiles_tract/2020/RI

fetching zip from https://www2.census.gov/geo/tiger/TIGER2020/TRACT/tl_2020_44_tract.zip...success
Get Rhode Island 2020 shapefiles_bg.../home/jupyter/raw_data/shapefiles_bg/2020/RI

fetching zip from https://www2.census.gov/geo/tiger/TIGER2020/BG/tl_2020_44_bg.zip...success
Get Rhode Island 2010 shapefiles_tract.../home/jupyter/raw_data/shapefiles_tract/2010/RI

fetching zip from https://www2.census.gov/geo/tiger/TIGER2010/TRACT/2010/tl_2010_44_tract10.zip...success
Get Rhode Island 2010 shapefiles_bg.../home/jupyter/raw_data/shapefiles_bg/2010/RI

fetching zip from https://www2.census.gov/geo/tiger/TIGER2010/BG/2010/tl_2010_44_bg10.zip...success
Get Texas 2020 shapefiles_tract.../home/jupyter/raw_data/shapefiles_tract/2020/TX

fetching zip from https://www2.census.gov/geo/tiger/TIGER2020/TRACT/tl_2020_48_tract.zip...success
Get Texas 2020 shapefiles_bg.../home/jupyter/raw_data/shapefiles_bg/2020

# old

In [ ]:
def get_geo(state, yr, level):
    yr = str(yr)
    if level == 'blockgroup':
        a = 'bg'
    elif level == 'block':
        a = 'tabblock'
    else:
        a = level

    path = root_path / f"geo/{level}/{yr}/{state['abbr']}"
    path.mkdir(exist_ok=True, parents=True)
    try:
        geo = gpd.read_file(path)
    except:
        %cd {path}
        url = f"https://www2.census.gov/geo/tiger/TIGER{yr}/{a.upper()}"
        
        if yr == '2010':
            url += '/2010'
        elif yr == '2020' and level == 'block':
            url += '20'
        url += f"/tl_{yr}_{state['fips']}_{a}{yr[-2:]}"
        if yr == '2020' and level in ['blockgroup', 'tract']:
            url = url[:-2]
        url += '.zip'
        ! wget -O geo.zip {url}
        ! unzip geo.zip
        ! rm geo.zip
        geo = gpd.read_file(path)

    geo = lower_cols(geo)[['geoid', 'aland', 'geometry']].apply(lambda p: orient(p, -1))
    geo.columns = ['geo_id', 'geometry']

    table_id = f"{proj_id}.geo.{level}-{yr}-{state['abbr']}"
    job = bqclient.load_table_from_dataframe(geo.to_wkt(), table_id)  # Make an API request.
    job.result()
    return geo


for i, state in states.iterrows():
    print(state['name'])
    for level in ['tract', 'blockgroup', 'block']:
        print(level)
        for yr in [2010, 2020]:
            print(yr)
            geo = get_geo(state, yr, level)

In [ ]:
def read_geo(state, yr, level):
    table_id = f"{proj_id}.geo.{level}-{yr}-{state['abbr']}"
    qry = f"""
        select
            *
        from
            {table_id}
        """
#     job = bqclient.load_table_from_dataframe(geo.to_wkt(), table_id)  # Make an API request.
    
#     job.result()  # Wait for the job to complete.
    bgs = bqclient.query(qry).result().to_dataframe()
    bgs['geometry'] = gpd.GeoSeries.from_wkt(bgs['geometry'])
    return bgs
#         table = bqclient.get_table(table_id)  # Make an API request.

state = states.iloc[0]
geo = get_geo(state, yr, level)
geo